In [1]:
%load_ext pretty_jupyter

In [2]:
# -.-|m { input: false, output: false, input_fold: show}

import tomlkit
import scanpy as sc
import anndata as ad

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rc_context
import patchworklib as pw
import seaborn.objects as so

from typing import List, Dict

from IPython.display import display
from os import walk, path, mkdir, listdir, cpu_count
import session_info
import logging
from utils.preprocessing_funcs import *

logging.basicConfig(level=logging.ERROR)
sc.set_figure_params(dpi = 100, dpi_save = 300, format= "png")
sc.settings.n_jobs = int(cpu_count() * 0.8)


<Figure size 100x100 with 0 Axes>

In [18]:
# -.-|m { input: false, output: false, input_fold: show}
"in the following cell, override the default pipeline parameters if needed"

#QC Params
CONCAT_SAMPLES: bool = True         # Concatenate all samples in one object, default: true
NMADS: int = 5                      # Number of median absolute deviations for read and gene counts.    
NMADS_MITO: int = 3                 # Number of median absolute deviations for mitochondrial genes percentage.
CORRECT_AMBIENT_RNA: bool = False   # Correct ambient RNA, uses DecontX, Currently causes multiple erros.
FILTER_DOUBLETS: bool = False        # Filter doublets using Scrublet
CELL_CYCLE_SCORE: bool = True       # Calculate cell cycle scores, based on scanpy implementation.
REGRESS: bool = False               # Regress out unwanted variables. Not recommended.
VARS_TO_REGRESS: List[str] = []     # list of regress (pct_counts_mt, pct_counts_ribo).



# QC Dict
"""
Can either be a flat-dict for global threshold or a dict of dicts for each sample 
Entries are in the form of col_name: [low, high] Ex: 'mt_pct':[0, 20]
"""

qc_dict = {}

# MODIFY_ME, Specifiy only after running diagnotics on the samples
# global config
qc_dict: Dict[str, List] | Dict[str, List] = {
    "pct_counts_mt": [0, 10],
    "pct_counts_rb": [0, 10],
    "n_genes_by_counts": [1500, 9000],
    "total_counts": [3_000, 40_000]
}


# # per-sample config
# qc_dict: Dict[str, List] | Dict[str, List] = {
#     "sample1": {
#         "pct_counts_mt": [0, 10],
#         "pct_counts_rb": [0, 10],
#         "n_genes_by_counts": [1500, 9000],
#         "total_counts": [3_000, 40_000]
#     },
#     "sample2": {
#         "pct_counts_mt": [0, 10],
#         "pct_counts_rb": [0, 10],
#         "n_genes_by_counts": [1500, 9000],
#         "total_counts": [3_000, 40_000]
#     },
#     "sample3": {
#         "pct_counts_mt": [0, 10],
#         "pct_counts_rb": [0, 10],
#         "n_genes_by_counts": [1500, 9000],
#         "total_counts": [3_000, 40_000]
#     },
#     "sample4": {
#         "pct_counts_mt": [0, 10],
#         "pct_counts_rb": [0, 10],
#         "n_genes_by_counts": [1500, 9000],
#         "total_counts": [3_000, 40_000]
#     }
# }


In [4]:
## Pipeline parameters

with open("../config.toml", "r") as f:
    config = tomlkit.parse(f.read())

In [5]:
#Directories 
ROOT_DIR = config["basic"]["ANALYSIS_DIR"]
DIR_SAVE = path.join(ROOT_DIR, config["basic"]["DIR_SAVE"])
DIR_samples = config["basic"]["DIR_SAMPLES"]

#Basic information
TECHNOLOGY: str = config["basic"]["TECHNOLOGY"]
ORGANISM: str = config["basic"]["ORGANISM"]
AUTODISCOVER: bool = config["basic"]["auto_find"]
samples: Dict[str, str] = config["basic"]["samples"]

sample_components = inputs[TECHNOLOGY]["files"]
black_list = inputs[TECHNOLOGY]["black_list"]
read_function = inputs[TECHNOLOGY]["function"]
raw_name = inputs[TECHNOLOGY]["raw_name"]

In [6]:
# Diagnosic pipeline
## Reading files

if AUTODISCOVER and len(samples) == 0:
    files = walk(DIR_samples)
    for root, dir, files in files:
        if len(set(sample_components).difference(set(files))) == 0 and path.basename(root) != raw_name:
            samples[get_sample_name(root, black_list, 5)] = root
else:
    samples = config["basic"]["samples"]
    if len(samples) > 0:
        pass
    else:
        raise RuntimeError("No samples paths were provided, provide sample paths as a dictionary in 'config.toml'")

# Samples (auto-discovered or manually added)

In [7]:
pd.DataFrame(samples,  index = ["sample path"]).T

,sample path
sample4,../test/sample4/filtered_feature_bc_matrix
sample3,../test/sample3/filtered_feature_bc_matrix
sample2,../test/sample2/filtered_feature_bc_matrix
sample1,../test/sample1/filtered_feature_bc_matrix


In [8]:
adatas = {}
raw_h5 = {}
for sample_id, filename in samples.items():
    sample_adata = read_function(filename)
    sample_adata.var_names_make_unique()
    adatas[sample_id] = sample_adata


if TECHNOLOGY == "10x" and CORRECT_AMBIENT_RNA:
    for sample_id, filename in samples.items():
        files = listdir(path.dirname(filename))
        raw_file = [file for file in files if "raw_feature_bc_matrix" in file and ".h5" in file]
        if len(raw_file) == 1:
            adata_raw = sc.read_10x_h5(path.join(path.dirname(filename), raw_file[0]))
        else:
            raise ValueError("No/Multiple raw files meeting condition were found")

        adata_raw.var_names_make_unique()
        raw_h5[sample_id] = adata_raw


if CONCAT_SAMPLES:
    adata = ad.concat(adatas, label="sample", join="outer", merge="same")
    adata.obs_names_make_unique()
    del samples

if CONCAT_SAMPLES and TECHNOLOGY == "10x" and CORRECT_AMBIENT_RNA:
    adata_raw = ad.concat(raw_h5, label="sample", join="outer", merge="same")
    adata_raw.obs_names_make_unique()
    del raw_h5


In [9]:
## Adding quality metrics


"""**TODO:**
- [ ] Make the Oulier function parameterized on sample
"""

mt_features = qc_features_fac[ORGANISM]["mito"]
rb_features = qc_features_fac[ORGANISM]["ribo"]
hb_features = qc_features_fac[ORGANISM]["hb"]


# mitochondrial genes, "MT-" for human, "Mt-" for mouse
adata.var["mt"] = adata.var_names.str.startswith(tuple(mt_features))
# ribosomal genes
adata.var["rb"] = adata.var_names.str.startswith(tuple(rb_features))
# hemoglobin genes
adata.var["hb"] = adata.var_names.str.contains(tuple(hb_features)[0]) #Only regex is accepted

sc.pp.calculate_qc_metrics(
    adata, qc_vars=["mt", "rb", "hb"], percent_top=[20],  inplace=True, log1p=True
)


In [19]:
#TODO: Add additional Critera for filtering based on Absolute thresholds, can it be per sample?
if len(qc_dict) > 0 and all(map(lambda x: isinstance(x, list), qc_dict.values())):
    reduce_outliers(adata, qc_dict)
elif len(qc_dict) > 0 and all(map(lambda x: isinstance(x, dict), qc_dict.values())):
    for sample in qc_dict.keys():
        reduce_outliers(adata, qc_dict[sample], sample)
else:
    adata.obs["total_counts_outlier"] = is_outlier(adata, "total_counts", NMADS)
    adata.obs["n_genes_by_counts_outlier"] = is_outlier(adata, "n_genes_by_counts", NMADS)
    adata.obs["pct_counts_mt_outlier"] = is_outlier(adata, "pct_counts_mt", NMADS_MITO)
    adata.obs["outlier"] = adata.obs["total_counts_outlier"] | adata.obs["n_genes_by_counts_outlier"] | adata.obs["pct_counts_mt_outlier"]

In [16]:

def reduce_outliers(adata: AnnData, variables: Dict[str, List], subset: str|None = None) -> pd.Series:
    outlier_dict = {}

    if subset is not None:
        for key in variables.keys():
            if key in adata.obs.columns:
                if len(variables[key]) == 2:
                    try:
                        adata.obs.loc[adata.obs["sample"] == subset, f"{key}_outlier"] = adata.obs.loc[adata.obs["sample"] == subset, key].lt(variables[key][0]) | \
                              adata.obs.loc[adata.obs["sample"] == subset, key].gt(variables[key][1])
                        outlier_dict[key] = adata.obs[f"{key}_outlier"] 
                    except:
                        adata.obs.loc[adata.obs["sample"] == subset, f"{key}_outlier"] = outlier_dict[key]
                        outlier_dict[key] = adata.obs[f"{key}_outlier"] 
                else:
                    raise ValueError("Provide a list of length 2 for the lower and upper bound of the QC-variable.")
            else:
                raise KeyError("the provided QC variable does not exist in the data, check the variable names again.")
            
        return reduce(lambda x, y: x or y, zip(outlier_dict.values()))

    for key in variables.keys():
        if key in adata.obs.columns:
            if len(variables[key]) == 2:
                outlier_dict[key] = adata.obs[key].lt(variables[key][0]) | adata.obs[key].gt(variables[key][1])
                adata.obs[f"{key}_outlier"] = outlier_dict[key]
            else:
                raise ValueError("Provide a list of length 2 for the lower and upper bound of the QC-variable.")
        else:
            raise KeyError("the provided QC variable does not exist in the data, check the variable names again.")

    return reduce(lambda x, y: x or y, zip(outlier_dict.values()))


In [24]:
adata.obs.loc[adata.obs["sample"] == "sample4", "pct_counts_rb_outlier"] = True
# for sample in qc_dict.keys():
#     print(adata.obs[["sample","total_counts_outlier"]].value_counts())
#     reduce_outliers(adata, qc_dict[sample], sample)


In [25]:
adata.obs.loc[adata.obs["sample"] == "sample4", "pct_counts_rb_outlier"] 

AAACCCAAGCCTGTCG-1    True
AAACCCACACAATGCT-1    True
AAACCCAGTTTACTGG-1    True
AAACCCATCATGAGGG-1    True
AAACCCATCTACACTT-1    True
                      ... 
TTTGTTGCAGGCTTGC-1    True
TTTGTTGGTTACCCTC-1    True
TTTGTTGGTTCGGCTG-1    True
TTTGTTGTCAATCTTC-1    True
TTTGTTGTCAGTGGGA-1    True
Name: pct_counts_rb_outlier, Length: 7365, dtype: bool

In [ ]:
## Ambient RNA correction
## TODO: Check if the Ambient RNA can be improved by using Batch information?

if CORRECT_AMBIENT_RNA and TECHNOLOGY == "10x":

    from os import system, remove, path
    import tempfile
    import urllib.request

    with tempfile.TemporaryDirectory(dir=".") as tmpdirname:

        # Workaround failure to install GenomeInfoDbData using pixi
        dn_path = path.join(tmpdirname, "GenomeInfoDbData_1.2.11.tar.gz")
        dn_url = "https://bioconductor.org/packages/3.18/data/annotation/src/contrib/GenomeInfoDbData_1.2.11.tar.gz"
        urllib.request.urlretrieve(dn_url, filename=dn_path)
        system(f"R CMD INSTALL {dn_path}")

        # Define paths for temporary files
        sce_path = path.join(tmpdirname, "sce.h5ad")
        raw_path = path.join(tmpdirname, "raw.h5ad")
        decontx_path = path.join(tmpdirname, "decontX.h5ad")
        
        # Save adata and adata_raw to the temporary directory
        adata.write_h5ad(sce_path)
        adata_raw.write_h5ad(raw_path)
        
        # Execute R scripts with temporary file paths
        system(f"Rscript ./utils/deconx.R -s {sce_path} -r {raw_path} -o {decontx_path}")

        # Read the result back from the temporary directory
        adata = sc.read_h5ad(decontx_path)

In [ ]:
## Doublet Detection

#TODO: Check real-life performance
#TODO: Check Interop with R to convert object to R & vice-versa 
if FILTER_DOUBLETS and TECHNOLOGY == "10x":
    sc.pp.scrublet(adata, batch_key="sample")

In [ ]:
## Cell cycle Scoring
# **Not reliable, do via interop later**

adata.layers["counts"] = adata.X.copy()
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)
sc.pp.scale(adata)


if CELL_CYCLE_SCORE:
    if ORGANISM in ["human", "mouse"]:

        s_genes = [x.strip() for x in open('../resources/s_genes.txt')]
        g2m_genes = [x.strip() for x in open('../resources/s_genes.txt')]


        if ORGANISM == "mouse":
            s_genes = human2mouse(s_genes)
            g2m_genes = human2mouse(g2m_genes)


        # Cell cycle scoring is not reliable and not similair to Seurat
        sc.tl.score_genes_cell_cycle(adata, s_genes=s_genes, g2m_genes=g2m_genes)
    else:
        logging.error('Organism must be either human or mouse.')

# Quality Plots

In [ ]:
# sc.set_figure_params(dpi=300, color_map="viridis_r")
# sc.settings.verbosity = 0

## Basic QC plots & metrics

In [ ]:
keys = ["n_genes_by_counts", "total_counts", "pct_counts_mt", "pct_counts_rb"]

if FILTER_DOUBLETS:
    keys = keys + ["doublet_score"]

if CELL_CYCLE_SCORE:
    keys = keys + ["S_score", "G2M_score"]

ncols = 2
nrows = len(keys) // ncols + len(keys) % ncols


figsize = 3
wspace = 0.5
fig, axs = plt.subplots(
    nrows=nrows,
    ncols=ncols,
    figsize=(ncols * figsize + figsize * wspace * (ncols - 1), nrows * figsize),
)

plt.subplots_adjust(wspace=wspace)
# Prevent the subplots from showing 
plt.close(fig)


for i, key in enumerate(keys):
    row = i // ncols
    col = i % ncols
    sc.pl.violin(adata, keys=[key], groupby="sample", stripplot=False, inner="box", ax=axs[row, col])

display(fig)


### Table of basic QC metrics

In [ ]:
df1 = adata.obs.groupby("sample")[keys].agg(["mean", "median"]).round(3)
df2 = adata.obs.groupby("sample")[["sample"]].agg(["size"])
pd.concat([df1, df2], axis =1)

## Histograms

In [ ]:
# Assuming 'df' is your DataFrame
df = adata.obs[keys]

# Determine the number of rows and columns for your subplot grid
n = len(df.columns)
ncols = 2
nrows = n // ncols + (n % ncols > 0)

# Create the subplots
wspace = 0.5
fig, axs = plt.subplots(
    nrows=nrows,
    ncols=ncols,
    figsize=(ncols * figsize + figsize * wspace * (ncols - 1), nrows * figsize + wspace * (nrows - 1) ),
)

plt.subplots_adjust(wspace=wspace, hspace=wspace)

# Flatten the axes
axs = axs.flatten()

outlier_dict = {}


# Plot a histogram on each subplot
for i, col in enumerate(df.columns):
    sns.histplot(data=df, x=col, ax=axs[i], bins=300)

    if col in ["n_genes_by_counts", "total_counts"]:
        max_line = min(df[col].agg("mean")+ median_abs_deviation(df[col]) * NMADS,  df[col].agg("max")) 
        min_line = max(df[col].agg("mean") - median_abs_deviation(df[col]) * NMADS, df[col].agg("min"))
        axs[i].axvline(x=max_line, color='red', linestyle='--', linewidth = 0.7)
        axs[i].axvline(x=min_line, color='red', linestyle='--', linewidth = 0.7)

        outlier_dict[col] = (min_line, max_line)

    elif col in ["pct_counts_mt"]:
        max_line = min(df[col].agg("mean")+ median_abs_deviation(df[col]) * NMADS_MITO,  df[col].agg("max")) 
        min_line = max(df[col].agg("mean") - median_abs_deviation(df[col]) * NMADS_MITO, df[col].agg("min"))
        axs[i].axvline(x=max_line, color='red', linestyle='--', linewidth = 0.7)
        axs[i].axvline(x=min_line, color='red', linestyle='--', linewidth = 0.7)

        outlier_dict[col] = (min_line, max_line)

# Remove any unused subplots
if len(df.columns) < nrows*ncols:
    for i in range(len(df.columns), nrows*ncols):
        fig.delaxes(axs[i])

plt.tight_layout()
plt.show()

## Filtering Thresholds

In [20]:
pd.DataFrame(outlier_dict, index= ("Min", "Max")).T

NameError: name 'outlier_dict' is not defined

## Number of outliers based on provided criteria

In [19]:
#TODO: Fix
df1 = pd.DataFrame(adata.obs[["total_counts_outlier", "n_genes_by_counts_outlier" ,"pct_counts_mt_outlier", "sample"]].value_counts(subset=["total_counts_outlier",  "sample"]))
df2 = pd.DataFrame(adata.obs[["n_genes_by_counts_outlier" , "sample"]].value_counts())
df3 = pd.DataFrame(adata.obs[["total_counts_outlier", "n_genes_by_counts_outlier" ,"pct_counts_mt_outlier", "sample"]].value_counts(subset=["pct_counts_mt_outlier",  "sample"]))

df = pd.concat([df1,df2,df3], axis=1)
df.columns = ["total_counts_outlier", "n_genes_by_counts_outlier", "pct_counts_mt_outlier"]
df


,,total_counts_outlier,n_genes_by_counts_outlier,pct_counts_mt_outlier
,sample,,,
False,sample4,5761,6393,7149
True,sample4,1604,972,216


## Cell filtering based on outlier function

In [ ]:
# Saving The object at the last step before subsseting
if path.exists(DIR_SAVE):
    adata.write_h5ad(path.join(DIR_SAVE, "raw_adata.h5ad"))
else:
    mkdir(DIR_SAVE)
    adata.write_h5ad(path.join(DIR_SAVE, "raw_adata.h5ad"))

# Cell Filtering based on threshold
adata = adata[(~adata.obs.outlier)].copy()

In [ ]:
df1 = adata.obs.groupby("sample")[keys].agg(["mean", "median"]).round(3)
df2 = adata.obs.groupby("sample")[["sample"]].agg(["size"])
pd.concat([df1, df2], axis =1)

## Scatter plots of confounders

In [ ]:
# Assuming 'df' is your DataFrame
df = adata.obs[keys+["sample"]]

# Determine the number of rows and columns for your subplot grid
n = len(df.columns)
ncols = 2
nrows = 2
figsize= 4

# Create the subplots
wspace = 0.5
fig, axs = plt.subplots(
    nrows=nrows,
    ncols=ncols,
    figsize=(ncols * figsize + figsize * wspace * (ncols - 1), nrows * figsize + wspace * (nrows - 1) ),
)

plt.subplots_adjust(wspace=wspace, hspace=wspace)

# Flatten the axes
axs = axs.flatten()

sns.scatterplot(df, x="total_counts", y="n_genes_by_counts", hue = "sample", alpha = 0.4, s = 6,  ax= axs[0])
sns.scatterplot(df, x="total_counts", y="pct_counts_mt", hue = "sample", alpha = 0.4, s = 6,  ax= axs[1])
sns.scatterplot(df, x="total_counts", y="pct_counts_rb", hue = "sample", alpha = 0.4, s = 6,  ax= axs[2])

if FILTER_DOUBLETS:
    sns.scatterplot(df, x="total_counts", y="doublet_score", hue = "sample", alpha = 0.4, s = 6,  ax= axs[3])

plt.tight_layout()
plt.show()



## DecontX contamination

In [ ]:

if CORRECT_AMBIENT_RNA and TECHNOLOGY == "10x":
    ax1 = pw.Brick(figsize=(6,6))
    ax2 = pw.Brick(figsize=(6,6))

    scatter = sns.scatterplot(pd.concat([adata.obsm["decontX_UMAP"], adata.obs], axis = 1), x = "DecontX_UMAP_1", y = "DecontX_UMAP_2", hue= "decontX_contamination" , s= 0.8, ax = ax1, palette="inferno")
    sns.move_legend(scatter, "center right", bbox_to_anchor=(1.1, 0.5), title=None, frameon=False)

    scatter = sns.scatterplot(pd.concat([adata.obsm["decontX_UMAP"], adata.obs], axis = 1), x = "DecontX_UMAP_1", y = "DecontX_UMAP_2", hue= "decontX_clusters" , s= 0.8, ax=ax2)
    sns.move_legend(scatter, "center right", bbox_to_anchor=(1.15, 0.5), title=None, frameon=False)

    ax12 = ax1+ax2
    display(ax12)

## Clustering after to cell filtering

In [ ]:
sc.pp.pca(adata, n_comps=20)
sc.pp.neighbors(adata)
sc.tl.leiden(adata, key_added="groups", flavor="igraph")
sc.tl.umap(adata)

In [ ]:
# -.-|m { input: false, output: true, input_fold: show}
keys = ["sample", "total_counts", "n_genes_by_counts", "pct_counts_mt"]

if FILTER_DOUBLETS:
    keys = keys + ["predicted_doublet"]

if CELL_CYCLE_SCORE:
    keys = keys + ["phase"]

figs = sc.pl.umap(adata, size= 7, color= keys, show=False, ncols = 2, color_map="inferno", sort_order=False, alpha = 0.8)

# Regression of Variables

In [ ]:
# - [ ] Add error handling if the vars to regress is empty or contain non-keys
if REGRESS:
    sc.pp.regress_out(adata, keys= VARS_TO_REGRESS)

# Ambient RNA 

In [ ]:
if CORRECT_AMBIENT_RNA:
    sns.violinplot(adata.obs, x = "decontX_clusters", y = "decontX_contamination")
    sns.stripplot(adata.obs, x = "decontX_clusters", y = "decontX_contamination", s = 1, c = "black")

In [ ]:
## Save Result
if path.exists(DIR_SAVE):
    adata.write_h5ad(path.join(DIR_SAVE, "adata.h5ad"))
else:
    mkdir(DIR_SAVE)
    adata.write_h5ad(path.join(DIR_SAVE, "adata.h5ad"))

# Session Information

In [ ]:
session_info.show()